In [11]:
import pandas as pd
from geopy.geocoders import Nominatim
from sklearn.cluster import KMeans
import requests
# ! pip install folium==0.5.0
import folium

In [3]:
df_latlng = pd.read_csv('https://raw.githubusercontent.com/ujoshidev/Coursera_Capstone/main/Week3/torronto-neuighbourhood-with-latlng.csv')
df_latlng

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern , Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill , Port Union , Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood , Morningside , West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village , St. Phillips , Martin Grov...",43.688905,-79.554724
101,M9V,Etobicoke,"South Steeles , Silverstone , Humbergate , Jam...",43.739416,-79.588437


#### Explore and cluster the neighborhoods in Toronto

In [5]:
#Fetching Torronto's lat lng. This is used to focus of the map on Torronto.
address = 'Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('Toronto coordinates are {}, {}.'.format(latitude, longitude))

Toronto coordinates are 43.6534817, -79.3839347.


In [24]:
# create map of Toronto using above lat long
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# adding markers
for lat, lng, borough, neighborhood in zip(df_latlng['Latitude'], df_latlng['Longitude'], df_latlng['Borough'], df_latlng['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [8]:
CLIENT_ID = 'T1LUMOHYFVR21KSZSVKTKIFO1DPONRPOL4PGKBLWSYTPNDNV' # your Foursquare ID
CLIENT_SECRET = 'JBKZMRN5WS2PBXLZDUMMLCYJIDZELUNSSP3QKM51GSHXXWB0' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 100
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)
radius = 500
#print(search_query + ' .... OK!')

url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
url

Your credentails:
CLIENT_ID: T1LUMOHYFVR21KSZSVKTKIFO1DPONRPOL4PGKBLWSYTPNDNV
CLIENT_SECRET:JBKZMRN5WS2PBXLZDUMMLCYJIDZELUNSSP3QKM51GSHXXWB0


'https://api.foursquare.com/v2/venues/search?client_id=T1LUMOHYFVR21KSZSVKTKIFO1DPONRPOL4PGKBLWSYTPNDNV&client_secret=JBKZMRN5WS2PBXLZDUMMLCYJIDZELUNSSP3QKM51GSHXXWB0&ll=43.6534817,-79.3839347&v=20180604&radius=500&limit=100'

In [15]:
#Sending the GET request 
res = requests.get(url).json()
res

venues = res['response']['venues']
#Tranforming venues into a dataframe
dataframe = pd.json_normalize(venues)
dataframe.head()

,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,location.crossStreet
0,5b193c42598e64002ca79b96,City of Toronto Civic Innovation Office,"[{'id': '4bf58dd8d48988d129941735', 'name': 'C...",v-1629974823,False,100 Queen St W,43.653454,-79.383952,"[{'label': 'display', 'lat': 43.653454, 'lng':...",3,M5H 2N2,CA,Toronto,ON,Canada,"[100 Queen St W, Toronto ON M5H 2N2, Canada]",NaN
1,4ad4c05ef964a5208ff620e3,Toronto City Hall,"[{'id': '4bf58dd8d48988d129941735', 'name': 'C...",v-1629974823,False,100 Queen St. W.,43.653140,-79.383967,"[{'label': 'display', 'lat': 43.65313989695342...",38,M5H 2N2,CA,Toronto,ON,Canada,"[100 Queen St. W. (at Bay St.), Toronto ON M5H...",at Bay St.
2,4c0121fd9a950f47fa9208c6,City Hall Podium Green Roof,"[{'id': '4bf58dd8d48988d15a941735', 'name': 'G...",v-1629974823,False,100 Queen Street West,43.653504,-79.383866,"[{'label': 'display', 'lat': 43.65350358617817...",6,NaN,CA,Toronto,ON,Canada,"[100 Queen Street West, Toronto ON, Canada]",NaN
3,4c41d47cd691c9b6fa4c8d0a,Kew Gardens Playground,"[{'id': '4bf58dd8d48988d1e7941735', 'name': 'P...",v-1629974823,False,Queen street and belfair ave,43.653225,-79.383185,"[{'label': 'display', 'lat': 43.65322456230742...",66,NaN,CA,Toronto,ON,Canada,"[Queen street and belfair ave, Toronto ON, Can...",NaN
4,4c093ee0340720a153728493,City Hall Council Chambers,"[{'id': '4bf58dd8d48988d129941735', 'name': 'C...",v-1629974823,False,NaN,43.651827,-79.383949,"[{'label': 'display', 'lat': 43.65182710471462...",184,NaN,CA,NaN,NaN,Canada,[Canada],NaN


In [21]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]
print(dataframe_filtered.shape)
dataframe_filtered.head()

(65, 15)


,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,crossStreet,id
0,City of Toronto Civic Innovation Office,City Hall,100 Queen St W,43.653454,-79.383952,"[{'label': 'display', 'lat': 43.653454, 'lng':...",3,M5H 2N2,CA,Toronto,ON,Canada,"[100 Queen St W, Toronto ON M5H 2N2, Canada]",NaN,5b193c42598e64002ca79b96
1,Toronto City Hall,City Hall,100 Queen St. W.,43.653140,-79.383967,"[{'label': 'display', 'lat': 43.65313989695342...",38,M5H 2N2,CA,Toronto,ON,Canada,"[100 Queen St. W. (at Bay St.), Toronto ON M5H...",at Bay St.,4ad4c05ef964a5208ff620e3
2,City Hall Podium Green Roof,Garden,100 Queen Street West,43.653504,-79.383866,"[{'label': 'display', 'lat': 43.65350358617817...",6,NaN,CA,Toronto,ON,Canada,"[100 Queen Street West, Toronto ON, Canada]",NaN,4c0121fd9a950f47fa9208c6
3,Kew Gardens Playground,Playground,Queen street and belfair ave,43.653225,-79.383185,"[{'label': 'display', 'lat': 43.65322456230742...",66,NaN,CA,Toronto,ON,Canada,"[Queen street and belfair ave, Toronto ON, Can...",NaN,4c41d47cd691c9b6fa4c8d0a
4,City Hall Council Chambers,City Hall,NaN,43.651827,-79.383949,"[{'label': 'display', 'lat': 43.65182710471462...",184,NaN,CA,NaN,NaN,Canada,[Canada],NaN,4c093ee0340720a153728493


In [30]:
dataframe_filtered['categories'].value_counts()

Office                      8
General Entertainment       6
City Hall                   5
Music Venue                 4
Coffee Shop                 3
Playground                  2
Event Space                 2
Art Gallery                 2
Food Truck                  2
Convenience Store           2
Miscellaneous Shop          1
Snack Place                 1
Parking                     1
Nursery School              1
Restaurant                  1
Performing Arts Venue       1
Smoke Shop                  1
Physical Therapist          1
Bike Rental / Bike Share    1
Garden                      1
Rest Area                   1
Other Nightlife             1
Gym                         1
Other Great Outdoors        1
Government Building         1
Bubble Tea Shop             1
Yoga Studio                 1
Sushi Restaurant            1
Bank                        1
Plaza                       1
Other Event                 1
Library                     1
Farmers Market              1
Building  

In [37]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Torronto City Hall

# add a red circle marker to represent the Torronto City Hall
folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Torronto City Hall',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# adding nearby offices as blue circle markers
for lat, lng in zip(dataframe_filtered[dataframe_filtered.categories=='Office'].lat, dataframe_filtered[dataframe_filtered.categories=='Office'].lng ):
    folium.CircleMarker(
        [lat, lng],
        radius=6,
        color='blue',
        popup='Office',
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)


# adding nearby Coffee shopsplaces as yellow circle markers
for lat, lng in zip(dataframe_filtered[dataframe_filtered.categories=='Coffee Shop'].lat, dataframe_filtered[dataframe_filtered.categories=='Coffee Shop'].lng):
    folium.CircleMarker(
        [lat, lng],
        radius=6,
        color='yellow',
        popup='Food Joint',
        fill = True,
        fill_color='yellow',
        fill_opacity=0.6
    ).add_to(venues_map)

# adding nearby General Entertainment as violet circle markers
for lat, lng in zip(dataframe_filtered[dataframe_filtered.categories=='General Entertainment'].lat, dataframe_filtered[dataframe_filtered.categories=='General Entertainment'].lng ):
    folium.CircleMarker(
        [lat, lng],
        radius=6,
        color='violet',
        popup='General Entertainment',
        fill = True,
        fill_color='violet',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map